In [1]:
from pypdevs.simulator import Simulator
from MaterialFlow import Gen_LINE
import pandas as pd



__  Current Time:       0.00 __________________________________________ 


	INITIAL CONDITIONS in model <Test.buffer>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <Test.cell.LinC0.conveyor1>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <Test.cell.LinC0.conveyor2>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <Test.cell.LinC0.station1>
		Initial State: ready
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <Test.cell.LinC0.station2>
		Initial State: ready
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <Test.cell.LinC1.conveyor1>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <Test.cell.LinC1.conveyor2>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in 

In [2]:
import pandas as pd
df = pd.read_excel('C:/Users/kms_main/OneDrive/바탕 화면/DEVS_git/GBP_model-pypdevs-lib-/result2.xlsx')

print(df)


    size_x  size_y     Type Total Time NumWorker  position_x  position_y  \
0        1       1  Station      38.57         1           8           0   
1        3       2     Cell      34.44         3           8           1   
2        1       1  Station      33.48         1          10           3   
3        1       1  Station      15.61         1          10           4   
4        1       1  Station       9.93         1          10           5   
5        1       1   Buffer          -         -          10           6   
6        1       1   Buffer          -         -          10           7   
7        1       1   Buffer          -         -          10           8   
8        1       1   Buffer          -         -          10           9   
9        1       1   Buffer          -         -          10          10   
10       1       1   Buffer          -         -          10          11   
11       1       1   Buffer          -         -          10          12   
12       1  

In [3]:
el_type = df['Type'].tolist()
el_time = df['Total Time'].tolist()

station_num = 1
cell_num = 1
tmp = []
el_dict = {}
for i in range(len(el_type)):
    if el_type[i] == "Cell":
        name = "cell_" + str(cell_num)
        cell_num += 1
        tmp.append(el_type[i])
        
    elif el_type[i] == "Station":
        name = "station_" + str(station_num)
        station_num += 1
        el_dict[name] = [int(el_time[i]),0,0,0]
        tmp.append(el_type[i])

    else:
        tmp.append("Conveyor")

el_type = tmp
print(el_type)
print(el_dict)

el_type = ['Station', 'Station', 'Station', 'Station', 'Conveyor', 'Conveyor', 'Station', 'Station', 'Station']


['Station', 'Cell', 'Station', 'Station', 'Station', 'Conveyor', 'Conveyor', 'Conveyor', 'Conveyor', 'Conveyor', 'Conveyor', 'Conveyor', 'Conveyor', 'Cell', 'Station', 'Station', 'Station']
{'station_1': [38, 0, 0, 0], 'station_2': [33, 0, 0, 0], 'station_3': [15, 0, 0, 0], 'station_4': [9, 0, 0, 0], 'station_5': [30, 0, 0, 0], 'station_6': [65, 0, 0, 0], 'station_7': [15, 0, 0, 0]}


In [4]:
import itertools

el_type = ['Station', 'Station', 'Station', 'Station', 'Conveyor', 'Conveyor', 'Station', 'Station', 'Station','Cell']

unique_permutations = set(itertools.permutations(el_type))
for i in unique_permutations:
    print(i)

('Station', 'Station', 'Station', 'Conveyor', 'Station', 'Station', 'Conveyor', 'Cell', 'Station', 'Station')
('Station', 'Cell', 'Conveyor', 'Conveyor', 'Station', 'Station', 'Station', 'Station', 'Station', 'Station')
('Station', 'Station', 'Station', 'Cell', 'Conveyor', 'Station', 'Station', 'Station', 'Conveyor', 'Station')
('Station', 'Conveyor', 'Cell', 'Station', 'Station', 'Station', 'Station', 'Station', 'Conveyor', 'Station')
('Cell', 'Station', 'Station', 'Station', 'Conveyor', 'Conveyor', 'Station', 'Station', 'Station', 'Station')
('Station', 'Conveyor', 'Conveyor', 'Station', 'Station', 'Station', 'Cell', 'Station', 'Station', 'Station')
('Station', 'Cell', 'Station', 'Conveyor', 'Station', 'Station', 'Station', 'Station', 'Conveyor', 'Station')
('Conveyor', 'Station', 'Station', 'Cell', 'Conveyor', 'Station', 'Station', 'Station', 'Station', 'Station')
('Station', 'Station', 'Conveyor', 'Station', 'Station', 'Station', 'Station', 'Cell', 'Conveyor', 'Station')
('Station'

In [16]:

input_dicts = []
inum = 0
for perm in unique_permutations:
    if inum == 1:
        break
    inum += 1

    final = []
    
    merged_list = []
    merged_time = []
    merged_name = []
    previous_item = None
    
    cl_cnt = 1
    st_cnt = 1
    cv_cnt = 1
    for item in perm:
        if item == 'Conveyor' and previous_item == 'Conveyor':
            merged_time[-1] += 2
            continue
        merged_list.append(item)
        if item == "Station":
            __name = "station_"+str(st_cnt)
            st_cnt += 1
            merged_time.append(el_dict[__name])
            merged_name.append(__name)
            
        elif item == "Cell":
            __name = "cell_"+str(cl_cnt)
            merged_name.append(__name)
            merged_time.append(2)

        else:
            merged_time.append(2)
            __name = "conveyor_" + str(cv_cnt)
            cv_cnt += 1
            merged_name.append(__name)

        previous_item = item
    
    inputData = {}
    name = ["source","source_buffer"]
    atomic = ["Source","Buffer"]
    time = [2,None,]

    for i in range(len(merged_list)):
        name.append(merged_name[i])
        atomic.append(merged_list[i])
        time.append(merged_time[i])

    

    name.append("result")
    atomic.append("Drain")
    time.append(None)

    inputData["name"] = name
    inputData["type"] = atomic
    inputData["time"] = time

    for key, value in inputData.items():
        print(f"{key}: {value}")

    print("------------------------------------------")

    sim = Simulator(Gen_LINE("LinearLine",inputData=inputData))

    sim.setVerbose()
    sim.setTerminationTime(1000)
    sim.setClassicDEVS()

    sim.simulate()
    print(sim.model.result.count)
    print("------------------------------------------")
   
        

name: ['source', 'source_buffer', 'station_1', 'station_2', 'station_3', 'conveyor_1', 'station_4', 'station_5', 'conveyor_2', 'cell_1', 'station_6', 'station_7', 'result']
type: ['Source', 'Buffer', 'Station', 'Station', 'Station', 'Conveyor', 'Station', 'Station', 'Conveyor', 'Cell', 'Station', 'Station', 'Drain']
time: [2, None, [38, 0, 0, 0], [33, 0, 0, 0], [15, 0, 0, 0], 2, [9, 0, 0, 0], [30, 0, 0, 0], 2, 2, [65, 0, 0, 0], [15, 0, 0, 0], None]
------------------------------------------

__  Current Time:       0.00 __________________________________________ 


	INITIAL CONDITIONS in model <LinearLine.cell_1.LinC0.conveyor1>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <LinearLine.cell_1.LinC0.conveyor2>
		Initial State: empty
		Next scheduled internal transition at time inf


	INITIAL CONDITIONS in model <LinearLine.cell_1.LinC0.station1>
		Initial State: ready
		Next scheduled internal transition at time inf


	INITIAL CON